# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import requests
import json
from datetime import timedelta

from geopy import Nominatim

# Load Windmill Data

In [2]:
useful_columns = ['case_id','t_state','t_county','t_fips','p_name','p_year',
                  'p_tnum','p_cap','t_manu','t_model','t_cap','t_hh','t_rd',
                  't_rsa','t_ttlh','xlong','ylat']
                  
df = pd.read_csv('../../data/raw/usgs/usgs_data.csv',
                 engine='python',encoding='latin1',usecols=useful_columns)
# We want only New York data
df = df[df.t_state=='NY']
df.head()

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,xlong,ylat
1528,3075480,NY,Chautauqua County,36013,Arkwright Summit,2018.0,36,78.4,Vestas,V110-2.0,2200.0,95.0,110.0,9503.32,150.0,-79.230286,42.395222
1529,3075488,NY,Chautauqua County,36013,Arkwright Summit,2018.0,36,78.4,Vestas,V110-2.0,2200.0,95.0,110.0,9503.32,150.0,-79.245026,42.427578
1530,3075460,NY,Chautauqua County,36013,Arkwright Summit,2018.0,36,78.4,Vestas,V110-2.0,2200.0,95.0,110.0,9503.32,150.0,-79.222168,42.388699
1531,3075489,NY,Chautauqua County,36013,Arkwright Summit,2018.0,36,78.4,Vestas,V110-2.0,2000.0,95.0,110.0,9503.32,150.0,-79.252396,42.430599
1532,3075456,NY,Chautauqua County,36013,Arkwright Summit,2018.0,36,78.4,Vestas,V110-2.0,2000.0,95.0,110.0,9503.32,150.0,-79.232803,42.430202


### Compute Blade Lengths

In [3]:
df['t_blade_length'] = df['t_rd'] / 2

# Main Computation Functions

In [4]:
def TravelInfo(origin, destination):
    """
    orgin: (x,y) where x and y represent latitude and longitude respectively
    destination: (x,y) where x and y represent latitude and longitude respectively

    returns travel distance in mi
    """
    # Unpack the latitudes and longitudes
    (lat1, lon1) = origin
    (lat2, lon2) = destination
    # Make API call to OSRM (Open Source Routing Machine) for travel info
    r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false""")
    # Unpack desired info
    route_1 = json.loads(r.content)["routes"][0]
    distance = route_1['distance']
    duration = route_1['duration']
    # Print output
    # print(f"Driving Distance (m): {distance}")
    # print(f"Driving Time (s): {duration}")
    # print("--------------------------------------------")
    # print(f"Driving Distance (mi): {distance/1609}")
    # print(f"Driving Time: {str(timedelta(seconds=duration))}")
    return distance/1609

### Units of GHG Emissions

miles $\times$ tons $\times$ $\frac{\text{grams}}{\text{tons $\cdot$ mile}} = \text{grams} $ 

In [5]:
def PredictTransportEmission(Windturbine,Destination):
    """
    Predicts GHG emissions for transporting one windmill blade

    destination: (x,y) where x and y represent latitude and longitude respectively

    Windturbine is a row from the df with windmill info.
    ex: Windturbine = df.iloc[7]
    """
    # Assume flatbed is 16.2 meters, we need __ flatbeds for transport
    numb_trucks = np.ceil(Windturbine['t_blade_length']/16.2)
    # Transport Distance (mi)
    Origin = (Windturbine['ylat'],Windturbine['xlong'])
    transport_distance = TravelInfo(Origin, Destination)
    # Shipment weight (lbs) --------------------------------------------------------- This we can create a model for if we can find some data
    weight = 5200       # low estimate
    weight_tons = weight / 2000
    # Total # of ton miles
    ton_miles = weight_tons * transport_distance
    # Trucks emit 161.8 grams per ton-mile
    emission_factor = 161.8

    ghg_emissions = transport_distance * weight_tons * emission_factor
    print(f'We expect {ghg_emissions} grams of GHG emissions.')
    print(f'That is {ghg_emissions/1000} kilograms of GHG emissions;')
    print(f'or, {ghg_emissions/1000000000} kilotons. ')


# Sample

Let's take a random windmill.

In [6]:
Windmill = df.iloc[1000]
Windmill

case_id                      3021582
t_state                           NY
t_county              Wyoming County
t_fips                         36121
p_name            Noble Wethersfield
p_year                        2009.0
p_tnum                            84
p_cap                          126.0
t_manu                       GE Wind
t_model                     GE1.5-77
t_cap                         1500.0
t_hh                            80.0
t_rd                            77.0
t_rsa                        4656.63
t_ttlh                         118.6
xlong                     -78.230995
ylat                       42.614792
t_blade_length                  38.5
Name: 41757, dtype: object

Let's send it to Delaware park in Buffalo New York. (In reality we will need to route to reuse centers, then to park; but, for example, we can use this.)

In [7]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Delaware Park, Buffalo, NY')

In [8]:
PredictTransportEmission(
    Windmill,
    (location.latitude,location.longitude)
                         )

We expect 21286.6955997514 grams of GHG emissions.
That is 21.2866955997514 kilograms of GHG emissions;
or, 2.1286695599751403e-05 kilotons. 
